In [3]:
# 导入视觉处理库 OpenCV
import cv2
# 导入数据处理库 numpy
import numpy as np

In [2]:
# 导入线程库threading
import threading
# 导入python的标准库time，提供与时间相关的函数
import time

In [3]:
# 导入用于在Jupyter中显示视频流的display库
from IPython.display import display
# ipywidgets 是一个用于在 Jupyter Notebook 和 JupyterLab 中创建交互式控件（例如按钮、滑块、文本框、图像显示等）的库
import ipywidgets as widgets

In [4]:
# 创建用于在 Jupyter 中显示的图像控件
image_widget_original = widgets.Image(format='jpeg')
image_widget_undistorted = widgets.Image(format='jpeg')
display(widgets.HBox([image_widget_original, image_widget_undistorted]))

In [6]:
cap = cv2.VideoCapture(0)
success = cap.isOpened()
# 关闭自动白平衡
cap.set(cv2.CAP_PROP_AUTO_WB, 0)
# 设置白平衡值
cap.set(cv2.CAP_PROP_WB_TEMPERATURE, 1000)
while success:
    ret, frames = cap.read()
    frame_rgb = cv2.cvtColor(frames, cv2.COLOR_BGR2RGB)
    cv2.imshow("", frames)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


In [10]:
cap.set(cv2.CAP_PROP_WB_TEMPERATURE, 200)

False

In [25]:
cap = cv2.VideoCapture(0)

In [29]:
# 关闭自动白平衡
cap.set(cv2.CAP_PROP_AUTO_WB,0)

False

In [26]:
cap.isOpened()

True

In [19]:
import cv2
import numpy as np

def simple_white_balance(img):
    """
    简单的完美反射白平衡算法。

    Args:
        img: 输入的彩色图像 (BGR 格式的 NumPy 数组)。

    Returns:
        白平衡调整后的彩色图像 (BGR 格式的 NumPy 数组)。
    """
    # 将图像转换为浮点数类型，以便进行除法运算
    img_float = img.astype(np.float32) / 255.0

    # 计算每个像素的亮度 (可以简单地取 RGB 三通道的最大值)
    brightness = np.max(img_float, axis=2)

    # 找到亮度最高的像素点的索引
    max_brightness_index = np.argmax(brightness)
    y, x = np.unravel_index(max_brightness_index, brightness.shape)

    # 获取最亮像素点的 RGB 值
    max_r, max_g, max_b = img_float[y, x]

    # 如果最亮点不是白色，则进行白平衡调整
    if max_r > 0 and max_g > 0 and max_b > 0:
        # 计算缩放因子，使得最亮点变为白色 (R=1, G=1, B=1)
        scale_r = 1.0 / max_r
        scale_g = 1.0 / max_g
        scale_b = 1.0 / max_b

        # 应用缩放因子到每个通道
        balanced_img = np.zeros_like(img_float)
        balanced_img[:, :, 0] = img_float[:, :, 0] * scale_b
        balanced_img[:, :, 1] = img_float[:, :, 1] * scale_g
        balanced_img[:, :, 2] = img_float[:, :, 2] * scale_r

        # 将像素值限制在 [0, 1] 范围内
        balanced_img = np.clip(balanced_img, 0, 1)
    else:
        balanced_img = img_float

    # 将图像转换回 8 位整数类型
    balanced_img_uint8 = (balanced_img * 255).astype(np.uint8)

    return balanced_img_uint8

if True:
    # 打开 USB 摄像头 (通常摄像头索引为 0，如果有多个摄像头可以尝试 1, 2, ...)
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("无法打开摄像头")
        exit()

    cv2.namedWindow('Original Camera Feed', cv2.WINDOW_NORMAL)
    cv2.namedWindow('SimpleWB Balanced Feed', cv2.WINDOW_NORMAL)

    while True:
        # 读取一帧图像
        ret, frame = cap.read()

        if not ret:
            print("无法接收到帧 (stream end?). 退出 ...")
            break

        # 应用完美反射白平衡算法
        balanced_frame = simple_white_balance(frame)

        # 显示原始图像和白平衡调整后的图像
        cv2.imshow('Original Camera Feed', frame)
        cv2.imshow('SimpleWB Balanced Feed', balanced_frame)

        # 按下 'q' 键退出循环
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # 释放摄像头并关闭窗口
    cap.release()
    cv2.destroyAllWindows()